# Load libraries

In [42]:
import os
from tqdm import tqdm
import polars as pl
import glob

# Parameters

In [9]:
TEST_DIRECTORY = './data/onward/building-instinct-test-data'
SAVE_TEST = './private_onward'
name_file = 'test_private_onward'

# Unify time series

In [10]:
def group_series_single_dataframe(read_dir,save_dir,name_file):
    # PUT ALL TIME SERIES IN A SINGLE FILE
    ALL = []
    for fn in tqdm(glob.glob(f'{read_dir}/*.parquet')):
        tmp = pl.read_parquet(fn).rename({
            'out.electricity.total.energy_consumption': 'energy',
            'in.state': 'state'
        }).with_columns([
            pl.col('energy').cast(pl.Float32).alias('energy'),
            pl.col('timestamp').dt.cast_time_unit("ms").dt.replace_time_zone(None).alias('timestamp')#.to_datetime("%Y-%m-%d %H:%M%#z")
        ])
        ALL.append(tmp)
    ALL = pl.concat(ALL)

    try:
        os.makedirs(f'{save_dir}')
    except OSError:
        pass

    ALL.write_parquet(f'{save_dir}/{name_file}.parquet')
    return ALL

In [11]:
group_series_single_dataframe(TEST_DIRECTORY, SAVE_TEST, name_file)

100%|██████████| 1440/1440 [00:02<00:00, 645.56it/s]


timestamp,energy,state,bldg_id
datetime[ms],f32,str,i64
2018-01-01 00:15:00,14.847511,"""NY""",498
2018-01-01 00:30:00,14.887037,"""NY""",498
2018-01-01 00:45:00,14.968305,"""NY""",498
2018-01-01 01:00:00,15.069287,"""NY""",498
2018-01-01 01:15:00,15.093472,"""NY""",498
…,…,…,…
2018-12-31 23:00:00,3.830655,"""MS""",502
2018-12-31 23:15:00,3.296935,"""MS""",502
2018-12-31 23:30:00,4.103808,"""MS""",502


# Extract features and save

In [12]:
os.system('python ./src/preprocess.py --read_dir ./private_onward --save_dir ./private_onward/data_ready')

READ_FOLDER ./private_onward
SAVE_FOLDER ./private_onward/data_ready
././private_onward/test_private_onward.parquet
././private_onward/test_private_onward.parquet -> Already done
././private_onward/test_private_onward.parquet -> Already done
././private_onward/test_private_onward.parquet -> Already done
././private_onward/test_private_onward.parquet -> Already done


0

# Make inference and save submission

In [20]:
os.system('python inference.py --read_dir ./private_onward/data_ready')

Reading -> energy
Reading -> energy_div_1
Reading -> energy_diff_abs_1
Reading -> energy_diff_abs_2
Dataset size (1440, 2830)


0